## FPGA Libraries

### Time testing for a very heavy program on CPU vs on GPU

In [ ]:
!pip install ccxt websockets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 26.4 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.14
    Uninstalling aiohttp-3.11.14:
      Successfully uninstalled aiohttp-3.11.14


In [ ]:
from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1
config.CUDA_LOW_OCCUPANCY_WARNINGS = 0

In [ ]:
import locale
print(locale.getpreferredencoding())


UTF-8


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [ ]:
!curl ipinfo.io


{
  "ip": "35.187.241.184",
  "hostname": "184.241.187.35.bc.googleusercontent.com",
  "city": "Singapore",
  "region": "Singapore",
  "country": "SG",
  "loc": "1.2897,103.8501",
  "org": "AS396982 Google LLC",
  "postal": "018989",
  "timezone": "Asia/Singapore",
  "readme": "https://ipinfo.io/missingauth"
}

## Scalping strategy CPU vs GPU

In [ ]:
import ccxt
from prettytable import PrettyTable
import os

class FindTopGainers():
    def __init__(self):
        self.exchange = ccxt.binance()
        self.initial_gains = {}
        self.data = {}

    def find_top_gainers(self):
        tickers = self.exchange.fetch_tickers()
        filtered_tickers = [ticker for ticker in tickers.values() if ticker['percentage'] is not None]
        sorted_tickers = sorted(filtered_tickers, key=lambda x: x['percentage'], reverse=True)
        table = PrettyTable()
        table.field_names = ["Symbol", "Price", "Percentage Change"]
        for ticker in sorted_tickers[:20]:
            table.add_row([ticker['symbol'], ticker['last'], ticker['percentage']])
        print(table)

Fetch_top_tickers = FindTopGainers()
Fetch_top_tickers.find_top_gainers()

+-----------------+-----------+-------------------+
|      Symbol     |   Price   | Percentage Change |
+-----------------+-----------+-------------------+
|     NIL/USDT    |   0.909   |       127.25      |
|    NIL/FDUSD    |   0.911   |      127.182      |
|     NIL/USDC    |   0.909   |      126.683      |
|     NIL/TRY     |   34.62   |      125.832      |
|     NIL/BNB     | 0.0014568 |      125.616      |
|    AIXBT/USDT   |   0.1311  |       26.789      |
|    AIXBT/USDC   |   0.131   |       26.448      |
|    AIXBT/TRY    |    4.98   |       23.881      |
| 1000CHEEMS/USDC |  0.001848 |       22.791      |
| 1000CHEEMS/USDT |  0.001847 |       22.724      |
|     ANKR/BTC    |  2.5e-07  |       19.048      |
|    ANKR/USDT    |  0.02121  |       18.096      |
|    BOME/USDT    |  0.00173  |       16.498      |
|    BOME/USDC    |  0.001725 |       16.083      |
|     ANKR/TRY    |   0.8066  |       15.824      |
|    BOME/FDUSD   |  0.001729 |       15.73       |
|   RENDER/U

In [ ]:
import ccxt
from prettytable import PrettyTable
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor
from numba import cuda, float32
import numpy as np
import threading
import nest_asyncio
import websockets
import json

nest_asyncio.apply()

class FindTopGainers():
    def __init__(self):
        self.exchange = ccxt.binance()
        self.data = {}
        self.top_gainers = []
        self.lock = threading.Lock()

    async def fetch_recent_trades(self, symbol):
        uri = f"wss://stream.binance.com:9443/ws/{symbol.lower()}@trade"
        trades = []

        async with websockets.connect(uri) as websocket:
            end_time = time.time() + 60  # Collect trades for 1 minute
            while time.time() < end_time:
                response = await websocket.recv()
                trade = json.loads(response)
                trades.append(trade)

        return trades

    @staticmethod
    @cuda.jit
    def calculate_percentage_change_kernel(start_price, end_price, result):
        idx = cuda.grid(1)
        if idx < start_price.size:
            result[idx] = ((end_price[idx] - start_price[idx]) / start_price[idx]) * 100

    def calculate_percentage_change(self, trades):
        if not trades:
            return 0
        start_price = np.array([trades[0]['p']], dtype=np.float32)
        end_price = np.array([trades[-1]['p']], dtype=np.float32)
        result = np.zeros_like(start_price)

        threads_per_block = 128
        blocks_per_grid = (start_price.size + (threads_per_block - 1)) // threads_per_block

        FindTopGainers.calculate_percentage_change_kernel[blocks_per_grid, threads_per_block](start_price, end_price, result)

        percentage_change = result[0]  # Get the calculated percentage change
        return percentage_change

    async def find_top_gainers(self):
        tickers = self.exchange.fetch_tickers()
        symbols = [ticker['symbol'] for ticker in tickers.values() if ticker['percentage'] is not None and '/USDT' in ticker['symbol']]
        percentage_changes = []

        tasks = [self.fetch_recent_trades(symbol) for symbol in symbols]
        trades_list = await asyncio.gather(*tasks)

        for symbol, trades in zip(symbols, trades_list):
            percentage_change = self.calculate_percentage_change(trades)
            percentage_changes.append((symbol, tickers[symbol]['last'], percentage_change))

        sorted_tickers = sorted(percentage_changes, key=lambda x: x[2], reverse=True)
        with self.lock:
            self.top_gainers = sorted_tickers[:5]

        table = PrettyTable()
        table.field_names = ["Symbol", "Price", "Percentage Change (5 min)"]
        for ticker in self.top_gainers:
            table.add_row([ticker[0], ticker[1], ticker[2]])
        print(table)

    async def update_ticker_list(self):
        while True:
            await self.find_top_gainers()
            await asyncio.sleep(60)  # Wait for 1 minute before fetching again

    def run(self, duration_minutes):
        print("Starting to find top gainers...")
        loop = asyncio.get_event_loop()
        end_time = time.time() + duration_minutes * 60

        async def stop_loop_after_duration():
            while time.time() < end_time:
                await asyncio.sleep(1)
            for task in asyncio.all_tasks(loop):
                task.cancel()

        loop.run_until_complete(asyncio.gather(self.update_ticker_list(), stop_loop_after_duration()))

if __name__ == "__main__":
    strategy = FindTopGainers()
    strategy.run(duration_minutes=1)  # Run the algorithm for 1 minute

Starting to find top gainers...


CancelledError: 

In [ ]:
import ccxt
from prettytable import PrettyTable
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor

class FindTopGainers():
    def __init__(self):
        self.exchange = ccxt.binance()
        self.data = {}
        self.top_gainers = []

    def fetch_recent_trades(self, symbol):
        since = self.exchange.milliseconds() - 5 * 60 * 1000  # 5 minutes ago
        trades = self.exchange.fetch_trades(symbol, since=since)
        return trades

    def calculate_percentage_change(self, trades):
        if not trades:
            return 0
        start_price = trades[0]['price']
        end_price = trades[-1]['price']
        percentage_change = ((end_price - start_price) / start_price) * 100
        return percentage_change

    def find_top_gainers(self):
        tickers = self.exchange.fetch_tickers()
        symbols = [ticker['symbol'] for ticker in tickers.values() if ticker['percentage'] is not None and '/USDT' in ticker['symbol']]
        percentage_changes = []

        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(self.fetch_recent_trades, symbol): symbol for symbol in symbols}
            for future in futures:
                symbol = futures[future]
                trades = future.result()
                percentage_change = self.calculate_percentage_change(trades)
                percentage_changes.append((symbol, tickers[symbol]['last'], percentage_change))

        sorted_tickers = sorted(percentage_changes, key=lambda x: x[2], reverse=True)
        for ticker in sorted_tickers[:10]:
            if ticker[0] not in [gainer[0] for gainer in self.top_gainers]:
                self.top_gainers.append(ticker)

        table = PrettyTable()
        table.field_names = ["Symbol", "Price", "Percentage Change (5 min)"]
        for ticker in self.top_gainers:
            table.add_row([ticker[0], ticker[1], ticker[2]])
        print(table)

    def run(self, duration_minutes):
        end_time = time.time() + duration_minutes * 60
        while time.time() < end_time:
            self.find_top_gainers()
            print("Waiting for 1 minute before next fetch...")
            time.sleep(60)  # Wait for 1 minute before fetching again

if __name__ == "__main__":
    strategy = FindTopGainers()
    strategy.run(duration_minutes=1)  # Run the algorithm for 10 minutes

In [ ]:
import ccxt
import numpy as np
from numba import cuda, njit
import timeit
import csv
from datetime import datetime as dt, timedelta
import pytz
import asyncio
import aiohttp
import cupy as cp
import pandas as pd
import nest_asyncio
from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt
nest_asyncio.apply()
from openpyxl import Workbook

class data_fetcher():
    def __init__(self):
        self.exchange = ccxt.binance()
        self.initial_gains = {}
        self.data = {}

class SwingHigh():

    def __init__(self):
        self.exchange = ccxt.binance()
        self.initial_gains = {}
        self.data = {}
        self.order_numbers = {}
        self.shares_per_ticker = {}
        self.positions = {}
        self.portfolio_value = 1000  # Initial portfolio value
        self.fees = 0.001  # Trading fee (0.1%)
        self.portfolio_history = []  # To track portfolio value over time

    async def find_top_gainers(self):
        tickers = self.exchange.fetch_tickers()
        filtered_tickers = [ticker for ticker in tickers.values() if ticker['percentage'] is not None]
        sorted_tickers = sorted(filtered_tickers, key=lambda x: x['percentage'], reverse=True)
        table = PrettyTable()
        table.field_names = ["Symbol", "Price", "Percentage Change"]
        for ticker in sorted_tickers[:20]:
            table.add_row([ticker['symbol'], ticker['last'], ticker['percentage']])
        print(table)
        top_gainers = [ticker['symbol'] for ticker in sorted_tickers[:20]]
        return top_gainers

    async def convert_timestamp_ms_to_human_readable(self, timestamp_ms):
        timestamp_s = timestamp_ms / 1000.0
        dt_object = dt.fromtimestamp(timestamp_s)
        hong_kong_tz = pytz.timezone('Asia/Hong_Kong')
        dt_object = dt_object.astimezone(hong_kong_tz)
        return dt_object.strftime('%Y-%m-%d %H:%M:%S')

    async def get_data(self, symbol, since, timeframe='1m', limit=1000):
        all_data = []
        while True:
            data = self.exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit)
            if not data:
                break
            all_data.extend(data)
            since = data[-1][0] + 1  # Move to the next timestamp
            if len(data) < limit:
                break

        # Convert timestamps to human-readable format
        for row in all_data:
            row[0] = await self.convert_timestamp_ms_to_human_readable(row[0])

        return all_data

    async def dynamic_pricing(self, symbol, since, timeframe='1m'):
        data = await self.get_data(symbol, since, timeframe)

        if not data:
            human_readable_since = await self.convert_timestamp_ms_to_human_readable(since)
            print(f"No data fetched for {symbol} since {human_readable_since}")
            return []

        # Fetch the last price for the previous minute
        last_price_data = await self.get_data(symbol, int(dt.now().timestamp() * 1000) - 60000, timeframe)
        last_price = last_price_data[-1][4] if last_price_data else data[0][4]

        # Add an additional column named 'last_price' with the last price of the ticker
        for row in data:
            row.append(last_price)
            last_price = row[4]  # Update last price to current row's close price

        return data

    async def fetch_and_save_data(self, symbol, writer):
        user_defined_time_frame = int((dt.now(pytz.timezone('Asia/Hong_Kong')) - timedelta(hours=1)).timestamp() * 1000)
        fetched_data = await self.dynamic_pricing(symbol, user_defined_time_frame, timeframe='1m')

        # Convert the fetched data to a DataFrame
        df = pd.DataFrame(fetched_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'last_price'])

        # Write the DataFrame to a sheet named after the symbol
        df.to_excel(writer, sheet_name=symbol.replace('/', '_'), index=False)

    @staticmethod
    @cuda.jit
    def calculate_gains(initial_prices, current_prices, gains):
        idx = cuda.grid(1)
        if idx < initial_prices.size:
            gains[idx] = (current_prices[idx] - initial_prices[idx]) / initial_prices[idx] * 100

    @staticmethod
    @njit
    def process_data(initial_prices, current_prices):
        gains = np.zeros_like(initial_prices, dtype=np.float32)
        for i in range(initial_prices.size):
            gains[i] = (current_prices[i] - initial_prices[i]) / initial_prices[i] * 100
        return gains

    async def fetch_the_volatile_cryptocurrencies(self, hours):
        start_time = timeit.default_timer()
        top_gainers_list = await self.find_top_gainers()

        # Create a Pandas Excel writer using openpyxl as the engine
        with pd.ExcelWriter('data.xlsx', engine='openpyxl') as writer:
            with ThreadPoolExecutor(max_workers=10) as executor:
                futures = [executor.submit(asyncio.run, self.fetch_and_save_data(symbol, writer)) for symbol in top_gainers_list if '/USDT' in symbol]
                for future in as_completed(futures):
                    future.result()  # Wait for all futures to complete

            # Ensure at least one sheet is visible
            if not writer.book.sheetnames:
                writer.book.create_sheet("Sheet1")

    async def load_volatile_tickers_excel_file(self, file_path):
        try:
            # Load the Excel file
            excel_data = pd.ExcelFile(file_path)

            # Initialize lists to store the data
            volatile_tickers = []
            initial_prices = []
            current_prices = []
            symbols = []

            since = int((dt.now(pytz.timezone('Asia/Hong_Kong')) - timedelta(hours=24)).timestamp() * 1000) # CHANGE HERE AS WELL TO 1 HR

            # Iterate through each sheet in the Excel file
            for sheet_name in excel_data.sheet_names:
                # Load the sheet into a DataFrame
                df = pd.read_excel(file_path, sheet_name=sheet_name)

                # Check if the DataFrame is empty
                if df.empty:
                    continue

                # Extract the necessary information
                latest_price = df['last_price'].iloc[-1]
                close_price = df['close'].iloc[-1]

                # Append the data to the respective lists
                volatile_tickers.append({
                    'symbol': sheet_name,
                    'initial_price': latest_price,
                    'current_price': close_price,
                    '%change': (close_price - latest_price) / latest_price * 100,
                    'num_trades': 0  # Initialize num_trades with 0
                })
                initial_prices.append(latest_price)
                current_prices.append(close_price)
                symbols.append(sheet_name.split('_')[0])  # Assuming the symbol is the part before '_USDT'

            # Convert to CuPy arrays for GPU processing
            initial_prices = cp.array(initial_prices, dtype=cp.float32)
            current_prices = cp.array(current_prices, dtype=cp.float32)
            gains = cp.zeros_like(initial_prices)

            threads_per_block = 128
            blocks_per_grid = (initial_prices.size + (threads_per_block - 1)) // threads_per_block
            self.calculate_gains[blocks_per_grid, threads_per_block](initial_prices, current_prices, gains)

            # Copy gains back to CPU from GPU
            gains = cp.asnumpy(gains)

            for i, symbol in enumerate(symbols):
                gain = gains[i]
                edited_symbol = symbol + '/USDT'
                num_trades = self.exchange.fetch_trades(edited_symbol, since=since)
                volatile_tickers[i]['num_trades'] = len(num_trades)  # Update num_trades with the actual number of trades
                if gain >= 2:
                    self.initial_gains[symbol] = gain
                elif symbol in self.initial_gains and gain < self.initial_gains[symbol] * 0.95:
                    volatile_tickers = [ticker for ticker in volatile_tickers if ticker['symbol'] != symbol]
                    del self.initial_gains[symbol]

            volatile_tickers.sort(key=lambda x: x['%change'], reverse=True)
            with open('volatile_tickers.csv', 'w') as f:
                writer = csv.writer(f)
                writer.writerow(['symbol', 'initial_price', 'current_price', '%change', 'num_trades'])
                for ticker in volatile_tickers:
                    writer.writerow([ticker['symbol'], ticker['initial_price'], ticker['current_price'], ticker['%change'], ticker['num_trades']])
            return volatile_tickers

        except Exception as e:
            print(f"An error occurred while loading the Excel file: {e}")
            return None

    async def log_message(self, message):
        print(message)
        with open('backtest_log.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([dt.now(pytz.timezone('Asia/Hong_Kong')), message])

    async def get_position(self, symbol):
        return self.positions.get(symbol, False)

    async def get_last_price(self, symbol):
        try:
            excel_data = pd.ExcelFile('data.xlsx')
            df = pd.read_excel('data.xlsx', sheet_name=symbol.replace('/', '_'))
            if df.empty:
                return None
            current_price = df['last_price'].iloc[-1]
            return current_price
        except Exception as e:
            print(f"An error occurred while fetching the last price for {symbol}: {e}")
            return None

    async def sell_all(self, symbol, current_price, timestamp, entry_price, buy_timestamp, highest_price, rationale):
        if await self.get_position(symbol):
            shares = self.shares_per_ticker[symbol]
            sale_value = shares * current_price
            sale_value -= sale_value * self.fees  # Subtract fees
            self.portfolio_value += sale_value
            gain_loss_percentage = ((current_price - entry_price) / entry_price) * 100
            await self.log_message(f"Selling all for {symbol} at {current_price} on {timestamp} (Bought at {buy_timestamp}, Highest price: {highest_price}, Gain/Loss: {gain_loss_percentage:.2f}%, Rationale: {rationale})")
            self.positions[symbol] = False
            self.portfolio_history.append(self.portfolio_value)  # Track portfolio value
            print(f"Portfolio value after selling {symbol}: {self.portfolio_value}")

    async def run_backtest(self):
        start_time = timeit.default_timer()
        await self.log_message("Starting backtest")
        backtest_end_time = dt.now(pytz.timezone('Asia/Hong_Kong')) + timedelta(hours=24)  # Adjust the backtesting period as needed

        fetch_Caller = await self.fetch_the_volatile_cryptocurrencies(hours=24)  # Fetch for 1 HOUR
        volatile_tickers = await self.load_volatile_tickers_excel_file(file_path="data.xlsx")
        if volatile_tickers is None:
            print("No volatile tickers found.")
            return

        self.symbols = [ticker['symbol'] for ticker in volatile_tickers]

        # Allocate 30% to the highest volatility ticker and 70% to the rest
        if volatile_tickers:
            highest_volatility_ticker = volatile_tickers[0]
            highest_volatility_allocation = self.portfolio_value * 0.1
            rest_allocation = self.portfolio_value * 0.7 / (len(volatile_tickers) - 1) if len(volatile_tickers) > 1 else 0

        for ticker in volatile_tickers:
            symbol = ticker['symbol']
            initial_price_trading = ticker['initial_price']
            allocation = highest_volatility_allocation if symbol == highest_volatility_ticker['symbol'] else rest_allocation
            shares = allocation / initial_price_trading
            self.shares_per_ticker[symbol] = shares
            self.positions[symbol] = True
            self.data[symbol] = []  # Initialize the data list for the symbol
            self.portfolio_value -= allocation  # Subtract the allocated amount from the portfolio value
            await self.log_message(f"Bought {shares} coins of {symbol} at {initial_price_trading}")
            print(f"Portfolio value after buying {symbol}: {self.portfolio_value}")

        for symbol in self.symbols:
            df = pd.read_excel('data.xlsx', sheet_name=symbol.replace('/', '_'))
            if df.empty:
                continue
            entry_price = None
            buy_timestamp = None
            highest_price = None
            for index, row in df.iterrows():
                current_price = row['last_price']
                timestamp = row['timestamp']
                if entry_price is None:
                    entry_price = current_price
                    buy_timestamp = timestamp
                    highest_price = current_price
                self.data[symbol].append(current_price)
                if current_price > highest_price:
                    highest_price = current_price
                if current_price < highest_price * 0.995: #EDIT HERE FOR VOLATILITY CHECKS #1
                    rationale = f"Price dropped below 90% of highest price ({highest_price})"
                    await self.sell_all(symbol, current_price, timestamp, entry_price, buy_timestamp, highest_price, rationale)
                    break  # Stop processing this ticker if sold
                if current_price > entry_price * 1.1: # EDIT HERE FOR VOLATILITY CHECKS #2
                    rationale = f"Price increased by 5% from entry price ({entry_price})"
                    await self.sell_all(symbol, current_price, timestamp, entry_price, buy_timestamp, highest_price, rationale)
                    break  # Stop processing this ticker if sold
                entry_price = current_price  # Update entry price dynamically

        # Sell everything at the end of the backtest
        for symbol in self.symbols:
            if await self.get_position(symbol):
                current_price = await self.get_last_price(symbol)
                if current_price is None:
                    continue
                timestamp = dt.now(pytz.timezone('Asia/Hong_Kong')).strftime('%Y-%m-%d %H:%M:%S')
                entry_price = self.data[symbol][0] if self.data[symbol] else current_price
                buy_timestamp = df['timestamp'].iloc[0] if not df.empty else timestamp
                rationale = "End of backtest period"
                await self.sell_all(symbol, current_price, timestamp, entry_price, buy_timestamp, highest_price, rationale)

        # Calculate final portfolio value
        final_portfolio_value = self.portfolio_value
        await self.log_message(f"Final portfolio value: {final_portfolio_value}")

        # Plot the portfolio value over time
        self.plot_portfolio_history()

        elapsed = timeit.default_timer() - start_time
        print(f"Backtest completed in {elapsed:.2f} seconds.")

    def plot_portfolio_history(self):
        times = np.arange(len(self.portfolio_history))
        portfolio_values = np.array(self.portfolio_history)

        plt.figure(figsize=(10, 6))
        plt.plot(times, portfolio_values, label='Portfolio Value')
        plt.xlabel('Time')
        plt.ylabel('Portfolio Value')
        plt.title('Portfolio Value Over Time')
        plt.legend()
        plt.grid(True)
        plt.show()

if __name__ == "__main__":
    strategy = SwingHigh()
    asyncio.run(strategy.run_backtest())

ModuleNotFoundError: No module named 'ccxt'

In [ ]:
import ccxt
from datetime import datetime as dt
from datetime import timedelta
import csv
import time
import timeit
import pytz

#TODO - Find a fix for : Error fetching data for BTCST/USDT:USDT: binance {"code":-1122,"msg":"Invalid symbol status."}

class SwingHigh():
    def __init__(self):
        self.exchange = ccxt.binance()
        self.initial_gains = {}
        self.data = {}
        self.order_numbers = {}
        self.shares_per_ticker = {}
        self.positions = {}
        self.portfolio_value = 1000  # Initial portfolio value
        self.fees = 0.1/100  # Binance trading fee (0.1%)

    def fetch_the_volatile_cryptocurrencies(self, hours):
        hkt = pytz.timezone('Asia/Hong_Kong')
        now = dt.now(hkt)
        print(f"Fetching coin prices from {self.exchange} from {hours} hour(s) ago to now which is {now} HKT")
        since = int((now - timedelta(hours=hours)).timestamp() * 1000)
        markets = self.exchange.load_markets()
        #markets = ["BTC/USDT", "ETH/USDT"]

        #markets = ['PI/USDT']

        volatile_tickers = []

        for symbol in markets:
            if '/USDT' in symbol:
                try:
                    data = self.get_minute_data(symbol, since)
                    if data:
                        initial_price = data[0][1]  # Opening price hours ago
                        current_price = data[-1][4]  # Closing price now
                        gain = (current_price - initial_price) / initial_price * 100
                        num_trades = self.exchange.fetch_trades(symbol, since=since)

                        if gain >= 2:
                            volatile_tickers.append({
                                'symbol': symbol,
                                'initial_price': initial_price,
                                'current_price': current_price,
                                '%change': gain,
                                'num_trades': num_trades
                            })
                            self.initial_gains[symbol] = gain
                        elif symbol in self.initial_gains and gain < self.initial_gains[symbol] * 0.95:
                            volatile_tickers = [ticker for ticker in volatile_tickers if ticker['symbol'] != symbol]
                            del self.initial_gains[symbol]
                except ccxt.BaseError as e:
                    print(f"Error fetching data for {symbol}: {e}")

        volatile_tickers.sort(key=lambda x: x['%change'], reverse=True)
        with open('volatile_tickers.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(['symbol', 'initial_price', 'current_price', '%change', 'num_trades'])
            for ticker in volatile_tickers:
                writer.writerow([ticker['symbol'], ticker['initial_price'], ticker['current_price'], ticker['%change'], ticker['num_trades']])
        return volatile_tickers

    def get_minute_data(self, symbol, since):
        ohlcv = self.exchange.fetch_ohlcv(symbol, timeframe='1m', since=since)
        return ohlcv

    def log_message(self, message):
        print(message)
        with open('backtest_log.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([dt.now(), message])

    def get_position(self, symbol):
        return self.positions.get(symbol, False)

    #TODO - problematic logic for last price - should be the last price of the candle. ie if the price at 12.00 is 100 and at 12.01 is 101, the last price should be 101 and not at the dt.now price
    def get_last_price(self, symbol):
        return self.exchange.fetch_ticker(symbol)['last']

    def sell_all(self, symbol, entry_price):
        current_price = self.get_last_price(symbol)
        if self.get_position(symbol):
            dropping_price =  entry_price * 0.995
            higher_than_earlier_price = entry_price * 1.015
            if current_price < dropping_price or current_price >= higher_than_earlier_price:
                shares = self.shares_per_ticker[symbol]
                sale_value = shares * current_price
                sale_value -= sale_value * self.fees  # Subtract fees
                self.portfolio_value += sale_value
                self.log_message(f"Selling all for {symbol} at {current_price} ")
                self.positions[symbol] = False

    def run_backtest(self):
        volatile_tickers = self.fetch_the_volatile_cryptocurrencies(hours=24)
        self.symbols = [ticker['symbol'] for ticker in volatile_tickers]

        # Allocate 30% to the highest volatility ticker and 70% to the rest
        if volatile_tickers:
            highest_volatility_ticker = volatile_tickers[0]
            highest_volatility_allocation = self.portfolio_value * 0.3
            rest_allocation = self.portfolio_value * 0.7 / (len(volatile_tickers) - 1) if len(volatile_tickers) > 1 else 0

        for ticker in volatile_tickers:
            symbol = ticker['symbol']
            initial_price_trading = ticker['initial_price']
            allocation = highest_volatility_allocation if symbol == highest_volatility_ticker['symbol'] else rest_allocation
            shares = allocation / initial_price_trading
            self.shares_per_ticker[symbol] = shares
            self.positions[symbol] = True
            self.data[symbol] = []  # Initialize the data list for the symbol
            self.log_message(f"Bought {shares} coins of {symbol} at {initial_price_trading}")

        for _ in range(60):
            for symbol in self.symbols:
                if self.get_position(symbol):
                    current_price = self.get_last_price(symbol)
                    entry_price = self.data[symbol][0] if symbol in self.data and self.data[symbol] else current_price
                    self.data[symbol].append(current_price)
                    if current_price < entry_price * 0.995 or current_price >= entry_price * 1.015:
                        self.sell_all(symbol, entry_price)
            #time.sleep(60)  # Wait for 1 minute

        # Sell everything at the end of the backtest
        for symbol in self.symbols:
            if self.get_position(symbol):
                self.sell_all(symbol, self.data[symbol][0])

        # Calculate final portfolio value
        final_portfolio_value = 0
        for symbol in self.symbols:
            if symbol in self.shares_per_ticker:
                final_portfolio_value += self.shares_per_ticker[symbol] * self.get_last_price(symbol)
        final_portfolio_value -= final_portfolio_value * self.fees  # Subtract fees

        self.log_message(f"Final portfolio value: {final_portfolio_value}")
if __name__ == "__main__":
    strategy = SwingHigh()
    start_time = timeit.default_timer()
    strategy.run_backtest()
    elapsed = timeit.default_timer() - start_time
    print(f"Backtest completed in {elapsed:.2f} seconds.")

Fetching coin prices from Binance from 24 hour(s) ago to now which is 2025-03-19 00:47:15.335129+08:00 HKT
Error fetching data for BTCST/USDT:USDT: binance {"code":-1122,"msg":"Invalid symbol status."}
Bought 5865.102639296188 coins of 1000X/USDT:USDT at 0.05115
Bought 25.759348803662242 coins of BMT/USDT at 0.1461
Bought 27.310891583563524 coins of MUBARAK/USDT:USDT at 0.1378
Bought 6.2411954564097085 coins of API3/USDT at 0.603
Bought 6.247411786545574 coins of API3/USDT:USDT at 0.6024
Bought 2.2004565633017914 coins of BNX/USDT at 1.7103
Bought 26.262671739114122 coins of BMT/USDT:USDT at 0.1433
Bought 7.647715627342113 coins of UXLINK/USDT:USDT at 0.4921
Bought 17.938230982912557 coins of LUNA/USDT at 0.2098
Bought 17.938230982912557 coins of LUNA2/USDT:USDT at 0.2098
Bought 178.7008955467737 coins of 1000RATS/USDT:USDT at 0.02106
Bought 0.39231114981914456 coins of NMR/USDT:USDT at 9.593
Bought 0.39121006862942354 coins of NMR/USDT at 9.62
Bought 3.1309824128245043 coins of UMA/US

In [ ]:
import ccxt
import numpy as np
from numba import cuda, njit
import timeit
import csv
from datetime import datetime as dt, timedelta
import pytz
import asyncio
import aiohttp
import cupy as cp


class SwingHigh():
    def __init__(self):
        self.exchange = ccxt.binance()
        self.initial_gains = {}
        self.data = {}
        self.order_numbers = {}
        self.shares_per_ticker = {}
        self.positions = {}
        self.portfolio_value = 1000  # Initial portfolio value
        self.fees = 0.1/100  # Binance trading fee (0.1%)


    @staticmethod
    @cuda.jit
    def calculate_gains(initial_prices, current_prices, gains):
        idx = cuda.grid(1)
        if idx < initial_prices.size:
            gains[idx] = (current_prices[idx] - initial_prices[idx]) / initial_prices[idx] * 100

    @staticmethod
    @njit
    def process_data(initial_prices, current_prices):
        gains = np.zeros_like(initial_prices, dtype=np.float64)
        for i in range(initial_prices.size):
            gains[i] = (current_prices[i] - initial_prices[i]) / initial_prices[i] * 100
        return gains

    async def fetch_data(self, symbol, since):
        loop = asyncio.get_event_loop()
        async with aiohttp.ClientSession() as session:
            try:
                data = await loop.run_in_executor(None, self.get_minute_data, symbol, since)
                return data
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")
                return None

    async def fetch_all_data(self, symbols, since):
        tasks = [self.fetch_data(symbol, since) for symbol in symbols]
        return await asyncio.gather(*tasks)

    async def fetch_the_volatile_cryptocurrencies(self, hours):
        hkt = pytz.timezone('Asia/Hong_Kong')
        now = dt.now(hkt)
        print(f"Fetching coin prices from {self.exchange} from {hours} hour(s) ago to now which is {now} HKT")
        since = int((now - timedelta(hours=hours)).timestamp() * 1000)
        markets = self.exchange.load_markets()
        volatile_tickers = []

        initial_prices = []
        current_prices = []
        symbols = []

        # Fetch data asynchronously
        symbols_to_fetch = [symbol for symbol in markets if '/USDT' in symbol]
        #symbols_to_fetch = ['PI/USDT']
        data_list = await self.fetch_all_data(symbols_to_fetch, since)

        for i, symbol in enumerate(symbols_to_fetch):
            data = data_list[i]
            if data:
                initial_prices.append(data[0][1])  # Opening price hours ago
                current_prices.append(data[-1][4])  # Closing price now
                symbols.append(symbol)

        # Convert to CuPy arrays for GPU processing
        initial_prices = cp.array(initial_prices, dtype=cp.float32)
        current_prices = cp.array(current_prices, dtype=cp.float32)
        gains = cp.zeros_like(initial_prices)

        threads_per_block = 128
        blocks_per_grid = (initial_prices.size + (threads_per_block - 1)) // threads_per_block
        self.calculate_gains[blocks_per_grid, threads_per_block](initial_prices, current_prices, gains)

        # Copy gains back to CPU from GPU
        gains = cp.asnumpy(gains)

        for i, symbol in enumerate(symbols):
            gain = gains[i]
            num_trades = self.exchange.fetch_trades(symbol, since=since)
            if gain >= 2:
                volatile_tickers.append({
                    'symbol': symbol,
                    'initial_price': initial_prices[i],
                    'current_price': current_prices[i],
                    '%change': gain,
                    'num_trades': num_trades
                })
                self.initial_gains[symbol] = gain
            elif symbol in self.initial_gains and gain < self.initial_gains[symbol] * 0.95:
                volatile_tickers = [ticker for ticker in volatile_tickers if ticker['symbol'] != symbol]
                del self.initial_gains[symbol]

        volatile_tickers.sort(key=lambda x: x['%change'], reverse=True)
        with open('volatile_tickers.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(['symbol', 'initial_price', 'current_price', '%change', 'num_trades'])
            for ticker in volatile_tickers:
                writer.writerow([ticker['symbol'], ticker['initial_price'], ticker['current_price'], ticker['%change'], ticker['num_trades']])

        return volatile_tickers

    def get_minute_data(self, symbol, since):
        ohlcv = self.exchange.fetch_ohlcv(symbol, timeframe='1m', since=since)
        return ohlcv

    def log_message(self, message):
        print(message)
        with open('backtest_log.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([dt.now(), message])

    def get_position(self, symbol):
        return self.positions.get(symbol, False)

    def get_last_price(self, symbol):
        return self.exchange.fetch_ticker(symbol)['last']

    def sell_all(self, symbol, entry_price):
        current_price = self.get_last_price(symbol)
        if self.get_position(symbol):
            dropping_price = entry_price * 0.995
            higher_than_earlier_price = entry_price * 1.015
            if current_price < dropping_price or current_price >= higher_than_earlier_price:
                shares = self.shares_per_ticker[symbol]
                sale_value = shares * current_price
                sale_value -= sale_value * self.fees  # Subtract fees
                self.portfolio_value += sale_value
                self.log_message(f"Selling all for {symbol} at {current_price} ")
                self.positions[symbol] = False

    async def run_backtest(self):
        volatile_tickers = await self.fetch_the_volatile_cryptocurrencies(hours=24)
        self.symbols = [ticker['symbol'] for ticker in volatile_tickers]

        # Allocate 30% to the highest volatility ticker and 70% to the rest
        if volatile_tickers:
            highest_volatility_ticker = volatile_tickers[0]
            highest_volatility_allocation = self.portfolio_value * 0.3
            rest_allocation = self.portfolio_value * 0.7 / (len(volatile_tickers) - 1) if len(volatile_tickers) > 1 else 0

        for ticker in volatile_tickers:
            symbol = ticker['symbol']
            initial_price_trading = ticker['initial_price']
            allocation = highest_volatility_allocation if symbol == highest_volatility_ticker['symbol'] else rest_allocation
            shares = allocation / initial_price_trading
            self.shares_per_ticker[symbol] = shares
            self.positions[symbol] = True
            self.data[symbol] = []  # Initialize the data list for the symbol
            self.log_message(f"Bought {shares} coins of {symbol} at {initial_price_trading}")

        for _ in range(60):
            for symbol in self.symbols:
                if self.get_position(symbol):
                    current_price = self.get_last_price(symbol)
                    entry_price = self.data[symbol][0] if symbol in self.data and self.data[symbol] else current_price
                    self.data[symbol].append(current_price)
                    if current_price < entry_price * 0.995 or current_price >= entry_price * 1.015:
                        self.sell_all(symbol, entry_price)

        # Sell everything at the end of the backtest
        for symbol in self.symbols:
            if self.get_position(symbol):
                self.sell_all(symbol, self.data[symbol][0])

        # Calculate final portfolio value
        final_portfolio_value = 0
        for symbol in self.symbols:
            if symbol in self.shares_per_ticker:
                final_portfolio_value += self.shares_per_ticker[symbol] * self.get_last_price(symbol)
        final_portfolio_value -= final_portfolio_value * self.fees  # Subtract fees

        self.log_message(f"Final portfolio value: {final_portfolio_value}")

import nest_asyncio
nest_asyncio.apply()

if __name__ == "__main__":
    start_time = timeit.default_timer()
    strategy = SwingHigh()
    start_time = timeit.default_timer()
    asyncio.run(strategy.run_backtest())
    elapsed = timeit.default_timer() - start_time
    print(f"Backtest completed in {elapsed:.2f} seconds.")


Fetching coin prices from Binance from 24 hour(s) ago to now which is 2025-03-19 01:09:41.727770+08:00 HKT
Error fetching data for BTCST/USDT:USDT: binance {"code":-1122,"msg":"Invalid symbol status."}
Bought 2053.38818359375 coins of BMT/USDT at 0.1460999995470047
Bought 194.8536376953125 coins of 1000X/USDT:USDT at 0.05282999947667122
Bought 74.64915466308594 coins of MUBARAK/USDT:USDT at 0.1378999948501587
Bought 16.90331268310547 coins of API3/USDT at 0.609000027179718
Bought 16.91720199584961 coins of API3/USDT:USDT at 0.6085000038146973
Bought 72.13817596435547 coins of BMT/USDT:USDT at 0.14270000159740448
Bought 6.009759902954102 coins of BNX/USDT at 1.7129000425338745
Bought 20.863636016845703 coins of UXLINK/USDT:USDT at 0.4934000074863434
Bought 50.684974670410156 coins of BURGER/USDT at 0.20309999585151672
Bought 48.76417922973633 coins of LUNA/USDT at 0.2110999971628189
Bought 48.787288665771484 coins of LUNA2/USDT:USDT at 0.210999995470047
Bought 482.8385314941406 coins of